In [ ]:
from scipy.spatial.distance import cosine
import json
import pandas as pd
import scipy
import numpy as np
from scipy.stats import entropy

In [ ]:
Domains = ['Chess','Fashion', 'AW', 'Finance']

All_Features = ['types', 'instances', 'NOUN', 'VERB', 'NNP']

In [ ]:
def create_freq(dict_seed, what):
    freq = {}
    for i in dict_seed:
        if (len(dict_seed[i][what])>0):
            freq[i] = create_freq_true(dict_seed[i][what], what)
    return freq

In [ ]:
def create_centroid(l, freq):
    data = pd.DataFrame(freq)
    data = data.fillna(0)
    data_to_sum = data[l[1:]]
    print(l)
    values = data[l[0]]
    
    l = l[1:]
    for name in l:
        values += data_to_sum[name]
    return values/len(l)

In [ ]:
def create_freq_true(dictTypes, what):
    freq = {}
    for k in dictTypes:
        if what != 'types' and what != 'instances':
            if k.isalpha():
                freq[k] = dictTypes[k]
        else:
            freq[k] = dictTypes[k]
    count = np.sum(list(freq.values()))
    for k in freq:
        freq[k] = freq[k]/count
    return freq



In [ ]:
def evaluation_cosine(freq, name, centroid):
    values = {}
    for f in freq:
        if f != name:
            s = pd.DataFrame({f:freq[f], name:centroid})
            s = s.fillna(0)
            values[f] = cosine(s[f], s[name])
    return values

In [ ]:
def evaluate(centroid, good, bad):
    distances = ['cosine']
    results = {}
    results['cosine'] = evaluation_cosine(good, 'centroid', centroid)
    results['cosine'].update(evaluation_cosine(bad, 'centroid', centroid))
    values = pd.Series(results['cosine']).sort_values()
    correct = 0
    precision = {}
    for i in range(len(values)):
        if values.index[i] in good:
            correct+=1
            precision[i] = correct/(i+1)
        else:
            if i == 0:
                precision[i] = 0
            else:
                precision[i] = precision[i-1]
    return precision, values

create feature vectors for all features in all domains

In [ ]:
freq_domains = {}
for d in Domains:
    freq_domains[d] = {}
    for f in All_Features:
        dom = json.loads(open(d+'_'+f+'.json').read())
        dom2 = {}
        for j in dom:
            dom2[j.lower()] = dom[j]
        freq_domains[d][f] = create_freq(dom2, f)

In [ ]:
random = '' #insert file name random

In [ ]:
freq_random = {}
for r in random:
    freq_random[r] = {}
    for f in Features:
        freq_random[r][f] = ev.create_freq(json.loads(open(r+'_'+f+'.json').read()), f)

prendere solo accounts con più di 50 tweets

In [ ]:
bad = {}
for f in All_Features:
    bad[f] = {}
for name_file in random:
    print(name_file)
    data = json.loads(open(file_path+name_file+'_tweets.json').read())
    lens = {}
    for user in data:
        lens[user] = len(data[user])
    
    for feat in All_Features:
        for user in freq_random[name_file][feat]:
            if user in lens:
                if lens[user] > 50:
                    bad[feat][user] = freq_random[name_file][feat][user]

calculate precision

In [ ]:
all_precision = {}
for dom in Domains:
    all_precision[dom] = {}
    combination_name = dom+'_Combinations.json' #take the 50 random combination created to split seeds and candidates
    combination = json.loads(open(combination_name).read())
    for f in All_Features:
        precision = {}
        for j in range(10+len(bad)):
            precision[j] = 0
        k = list(freq_domains[dom][f].keys())
        for i in range(50):
            cent = create_centroid(combination[str(i)], freq_domains[dom][f])
            good = {}
            for name in k:
                if name not in combination[str(i)]:
                    good[name] = freq_domains[dom][f][name]
            prec, _ = evaluate(cent, good, bad)
            print(prec)
            for j in range(len(good)+len(bad)):
                    precision[j] += prec[j]
        for i in range(len(precision)):
            precision[i] /= 50
        all_precision[dom][f] = precision

In [ ]:
for dom in Domains:
    for f in All_Features:
        print('dom','f',all_precision[dom][f][10])